In [7]:
import lxml.html
import numpy as np
import pandas as pd
import re
import requests
import html

In [8]:
def get_people(people, tree, occupation):
    """ Retrieve the people using cssselect """
    for element in tree.cssselect('div[class*=\'tile\']'):
        try:
            person = {}
            person['name'] = element.cssselect('a img')[0].get('title').strip()
            if person['name'] in people.keys():
                people[person['name']]['occupation'] = people[person['name']]['occupation'] + ","+occupation
            else:
                person['link'] = 'http://'+element.cssselect('a')[0].get('href')[2:]
                # All person links have 'www'. Other links do not
                if 'www' not in person['link']:
                    break
                person['occupation'] = occupation
                person['image'] = element.cssselect('a img')[0].get('src')
                people[person['name']] = person
        except:
            pass
    return people

In [10]:
FAM_PPL_URL = "http://www.thefamouspeople.com/writers.php"
people = {}
for i in range(1,8):
    link = FAM_PPL_URL+"?page=%s" % i
    resp = requests.get(link)
    if resp.status_code != 200:
        raise Exception('GET '+link+' {}'.format(resp.status_code))
    tree = lxml.html.fromstring(resp.content)
    people = get_people(people, tree, 'author')
print('Found %s persons!' % len(people))

Found 693 persons!


In [ ]:
%%time
for elem in people:
    person = people[elem]
    try:
        tree = common.get_tree_from_url(person['link'])
        try:
            person['image'] = tree.cssselect('div[class*=\'fps-img\'] img')[0].get('src')
        except:
            print(person['link'])
            break
        try:
            person['description'] = tree.cssselect('p[class*=\'main_introduction_right\']')[0].text_content().strip()
        except:
            pass
        content = str(lxml.html.tostring(tree.cssselect('div[class*=\'fps-desc\']')[0]))
        content = re.sub('<a[^<]+?>', '', content.replace('\\r\\\n', '').replace('  ', ' ').replace('</a>', '').replace('<span>', '').replace('</span>', ''))
        for result in re.findall('<div class="quick_left">(.*?)<\/div>.*?<div class="quick_right">(.*?)<\/div>', content):
            key = result[0].strip().strip("\n")
            value = html.unescape(result[1].strip().strip('\n'))
            person[key] = value
    except:
        print("Retrieval failed for", person['name'])